In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

In [ ]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

In [ ]:
def load_data():
    """
        Load the data:
            - 14,034 images to train the network.
            - 3,000 images to evaluate how accurately the network learned to classify images.
    """
    
    datasets = ['/kaggle/input/intel-image-classification/seg_train/seg_train', '/kaggle/input/intel-image-classification/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output
                
            

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
train_labels.shape


In [ ]:
train_images,train_labels=shuffle(train_images,train_labels,random_state=25)

In [ ]:
train_images=train_images/255
test_images=test_images/255

In [ ]:
import pandas as pd

In [ ]:
_,train_count=np.unique(train_labels,return_counts=True)
_,test_count=np.unique(test_labels,return_counts=True)

In [ ]:
pd.DataFrame({'train':train_count,'test':test_count},index=class_names).plot.bar()
plt.show()

In [ ]:
plt.pie(train_count,labels=class_names,autopct='%1.1f%%')

In [ ]:
plt.pie(test_count,labels=class_names,autopct='%1.1f%%')

In [ ]:
index=np.random.randint(train_images.shape[0])
index

In [ ]:
def display_random_image(class_names, images, labels):
    """
        Display a random image from the images array and its correspond label from the labels array.
    """
    
    index = np.random.randint(images.shape[0])
    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title('Image #{} : '.format(index) + class_names[labels[index]])
    plt.show()

In [ ]:
display_random_image(class_names,train_images,train_labels)

In [ ]:
image = cv2.imread('/kaggle/input/intel-image-classification/seg_test/seg_test/sea/21568.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
image=cv2.resize(image,IMAGE_SIZE)
plt.imshow(image)
plt.imshow(train_images[9959])

**Creating a basic CNN model**

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(6,tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_images,train_labels,batch_size=128,epochs=20,validation_split=0.2)

In [ ]:
model.evaluate(test_images,test_labels)

In [ ]:
predictions=model.predict(test_images)

In [ ]:
pred_labels=[]
for i in range(len(predictions)):
    pred=np.argmax(predictions[i])
    pred_labels.append(pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels,pred_labels)*100

In [ ]:
display_random_image(class_names, test_images, pred_labels)

In [ ]:
CM=confusion_matrix(test_labels,pred_labels)

In [ ]:
sn.heatmap(CM,annot=True,xticklabels=class_names,yticklabels=class_names)

**Using Pretrained Model VGG Imagenet**

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
model = VGG16(weights='imagenet',include_top=False)

In [ ]:
train_features=model.predict(train_images)
test_features=model.predict(test_images)

In [ ]:
train_features.shape

In [ ]:
test_features.shape

In [ ]:
n_train,x,y,z=train_features.shape
n_test, x,y,z=test_features.shape

In [ ]:
(x,y,z)

In [ ]:
model2=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (x,y,z)),
    tf.keras.layers.Dense(50,activation = tf.nn.relu),
    tf.keras.layers.Dense(6,activation = tf.nn.softmax)
])

In [ ]:
model2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
History2=model2.fit(train_features,train_labels,batch_size=128,epochs=15,validation_split=0.2)

In [ ]:
model2.evaluate(test_features,test_labels)

In [ ]:
predictions=model2.predict(test_features)

In [ ]:
predictions[0]

In [ ]:
accuracy_score(test_labels,pred_labels)*100

In [ ]:
display_random_image(class_names,test_images,pred_labels)

In [ ]:
cm=confusion_matrix(test_labels,pred_labels)

In [ ]:
sn.heatmap(cm,annot=True,xticklabels=class_names,yticklabels=class_names)

**Ensemble Neural Network**

In [ ]:
np.random.seed(seed=1997)
n_estimators=10
max_samples=0.8
max_samples*=n_train
max_samples=int(max_samples)

In [ ]:
models=list()
random=np.random.randint(50,100,size=n_estimators)
for i in range(n_estimators):
    model=tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(x,y,z)),
        tf.keras.layers.Dense(random[i],activation=tf.nn.relu),
        tf.keras.layers.Dense(6,activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    models.append(model)

In [ ]:
models

In [ ]:
Histories=[]
for i in range(n_estimators):
    train_idx=np.random.choice(len(train_features),max_samples)
    Histories.append(models[i].fit(train_features[train_idx],train_labels[train_idx],epochs=10,batch_size=128,validation_split=0.1))

In [ ]:
predictions=[]
for i in range(n_estimators):
    predictions.append(models[i].predict(test_features))

In [ ]:
predictions

In [ ]:
predictions=np.array(predictions)

In [ ]:
predictions=predictions.sum(axis=0)

In [ ]:
pred_labels=predictions.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy=accuracy_score(test_labels,pred_labels)*100

In [ ]:
accuracy

**Fine Tuning VGG ImageNet**

In [ ]:
model=VGG16(weights='imagenet',include_top=False)

In [ ]:
from keras.models import Model

In [ ]:
model=Model(model.inputs,model.layers[-5].output)

In [ ]:
model.summary()

In [ ]:
train_features=model.predict(train_images)
test_features=model.predict(test_images)

In [ ]:
from keras.layers import Input,Dense,Conv2D,Activation,MaxPooling2D,Flatten

In [ ]:
model2=VGG16(weights='imagenet',include_top=False)

In [100]:
model2.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
input_shape=model2.layers[-4].get_input_shape_at(0)

In [ ]:
layer_input=Input(shape=(9,9,512))

In [101]:
x=layer_input

In [102]:
for layer in model2.layers[-4::1]:
    x=layer(x)
x=Conv2D(64,(3,3),activation='relu')(x)
x=MaxPooling2D(pool_size=(2,2))(x)
x=Flatten()(x)
x=Dense(100,activation='relu')(x)
x=Dense(6,activation='softmax')(x)
new_model=Model(layer_input,x)

In [103]:
new_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 9, 9, 512)         0         
_________________________________________________________________
block5_conv1 (Conv2D)        multiple                  2359808   
_________________________________________________________________
block5_conv2 (Conv2D)        multiple                  2359808   
_________________________________________________________________
block5_conv3 (Conv2D)        multiple                  2359808   
_________________________________________________________________
block5_pool (MaxPooling2D)   multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 64)          294976    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0   

In [104]:
new_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [106]:
History3=new_model.fit(train_features,train_labels,epochs=15,validation_split=0.2,batch_size=128)

Train on 11227 samples, validate on 2807 samples
Epoch 1/15
11227/11227 [==============================] - 5s 458us/step - loss: 0.2564 - accuracy: 0.9135 - val_loss: 0.2372 - val_accuracy: 0.9191
Epoch 2/15
11227/11227 [==============================] - 5s 418us/step - loss: 0.2083 - accuracy: 0.9281 - val_loss: 0.2629 - val_accuracy: 0.9116
Epoch 3/15
11227/11227 [==============================] - 5s 417us/step - loss: 0.1742 - accuracy: 0.9395 - val_loss: 0.2445 - val_accuracy: 0.9177
Epoch 4/15
11227/11227 [==============================] - 5s 418us/step - loss: 0.1505 - accuracy: 0.9461 - val_loss: 0.2479 - val_accuracy: 0.9152
Epoch 5/15
11227/11227 [==============================] - 5s 414us/step - loss: 0.1172 - accuracy: 0.9592 - val_loss: 0.2786 - val_accuracy: 0.9124
Epoch 6/15
11227/11227 [==============================] - 5s 413us/step - loss: 0.1076 - accuracy: 0.9621 - val_loss: 0.3375 - val_accuracy: 0.8978
Epoch 7/15
11227/11227 [==============================] - 5s 41

In [107]:
predictions=new_model.predict(test_features)

In [108]:
pred_labels=[]
for i in range(0,len(predictions)):
    pred_labels.append(np.argmax(predictions[i]))

In [109]:
accuracy_score(test_labels,pred_labels)

0.8893333333333333